# I.4 Dual Numbers

Int his chapter we introduce a mathematically beautiful  alternative to divided differences for computing
derivatives: _dual numbers_. As we shall see, these are a commutative ring that _exactly_ compute derivatives,
which when implemented in floating point give very high-accuracy approximations to derivatives. They
underpin forward-mode [automatic differentation](https://en.wikipedia.org/wiki/Automatic_differentiation).


Before we begin, we must be clear what a "function" is. Consider three possible scenarios:

1. _Black-box function_: this was considered last chapter where we can only input floating-point numbers
and get out a floating-point number.
2. _Generic function_: Consider a function that is a formula (or, equivalentally, a _piece of code_) 
that we can evaluate  on arbitrary types, including
custom types that we create. An example is a polynomial:
$$
p(x) = p_0 + p_1 x + ⋯ + p_n x^n
$$
which can be evaluated for $x$ in the reals, complexes, or any other ring.
 More generally, if we have a function defined in Julia that does not call any
C libraries it can be evaluated on different types. 
3. _Graph function_: The function is built by composing different basic "kernels" with known differentiability properties.
We won't consider this situation in this module, though it is the model used by Python machine learning toolbox's
like [PyTorch](https://pytorch.org) and [TensorFlow](http://tensorflow.org).




**Definition 1 (Dual numbers)** Dual numbers $𝔻$ are a commutative ring (over $ℝ$)
generated by $1$ and $ϵ$ such that $ϵ^2 = 0$.
Dual numbers are typically written as $a + b ϵ$ where $a$ and $b$ are real.

This is very much analoguous to complex numbers, which are a field generated by $1$ and ${\rm i}$ such that
${\rm i}^2 = -1$. Compare multiplication of each number type:
$$
\begin{align*}
(a + b {\rm i}) (c + d {\rm i}) &= ac + (bc + ad) {\rm i} + bd {\rm i}^2 = ac -bd + (bc + ad) {\rm i} \\
(a + b ϵ) (c + d ϵ) &= ac + (bc + ad) ϵ + bd ϵ^2 = ac  + (bc + ad) ϵ 
\end{align*}
$$
And just as we view $ℝ ⊂ ℂ$ by equating $a ∈ ℝ$ with $a + 0{\rm i} ∈ ℂ$,
we can view $ℝ ⊂ 𝔻$ by equating $a ∈ ℝ$ with $a + 0{\rm ϵ} ∈ 𝔻$.




## 1. Differentiating polynomials

Applying a polynomial to a dual number $a + b ϵ$ tells us the derivative at $a$:

**Theorem 1 (polynomials on dual numbers)** Suppose $p$ is a polynomial. Then
$$
p(a + b ϵ) = p(a) + b p'(a) ϵ
$$

**Proof**

It suffices to consider $p(x) = x^n$ for $n ≥ 1$ as other polynomials follow from linearity. We proceed by induction:
The case $n = 1$ is trivial. For $n > 1$ we have 
$$
(a + b ϵ)^n = (a + b ϵ) (a + b ϵ)^{n-1} = (a + b ϵ) (a^{n-1} + (n-1) b a^{n-2} ϵ) = a^n + b n a^{n-1} ϵ.
$$

∎

**Example 1 (differentiating polynomial)** Consider computing $p'(2)$ where
$$
p(x) = (x-1)(x-2) + x^2.
$$
We can use Dual numbers to differentiating, avoiding expanding in monomials
or rules of differentiating:
$$
p(2+ϵ) = (1+ϵ)ϵ + (2+ϵ)^2 = ϵ + 4 + 4ϵ = 4 + \underbrace{5}_{p'(2)}ϵ
$$



## 2. Differentiating other functions


We can extend real-valued differentiable functions to dual numbers in a similar manner.
First, consider a standard function with a Taylor series (e.g. ${\rm cos}$, ${\rm sin}$, ${\rm exp}$, etc.)
$$
f(x) = ∑_{k=0}^∞ f_k x^k
$$
so that $a$ is inside the radius of convergence. This leads naturally to a definition on dual numbers:
$$
\begin{align*}
f(a + b ϵ) &= ∑_{k=0}^∞ f_k (a + b ϵ)^k = f_0 + ∑_{k=1}^∞ f_k (a^k + k a^{k-1} b ϵ) = ∑_{k=0}^∞ f_k a^k +  ∑_{k=1}^∞ f_k k a^{k-1} b ϵ  \\
  &= f(a) + b f'(a) ϵ
\end{align*}
$$
More generally, given a differentiable function we can extend it to dual numbers:

**Definition 2 (dual extension)** Suppose a real-valued function $f$
is differentiable at $a$. If
$$
f(a + b ϵ) = f(a) + b f'(a) ϵ
$$
then we say that it is a _dual extension at_ $a$.

Thus, for basic functions we have natural extensions:
$$
\begin{align*}
\exp(a + b ϵ) &:= \exp(a) + b \exp(a) ϵ \\
\sin(a + b ϵ) &:= \sin(a) + b \cos(a) ϵ \\
\cos(a + b ϵ) &:= \cos(a) - b \sin(a) ϵ \\
\log(a + b ϵ) &:= \log(a) + {b \over a} ϵ \\
\sqrt{a+b ϵ} &:= \sqrt{a} + {b \over 2 \sqrt{a}} ϵ \\
|a + b ϵ| &:= |a| + b\, {\rm sign} a\, ϵ
\end{align*}
$$
provided the function is differentiable at $a$. Note the last example does not have
a convergent Taylor series (at 0) but we can still extend it where it is differentiable.

Going further, we can add, multiply, and compose such functions:

**Lemma 1 (product and chain rule)**
If $f$ is a dual extension at $g(a)$ and $g$
is a dual extension at $a$, then $q(x) := f(g(x))$ is a dual extension at $a$.
If $f$ and $g$ are dual extensions at $a$ then 
$r(x) := f(x) g(x)$ is also dual extensions at $a$. In other words:
$$
\begin{align*}
q(a+b ϵ) &= q(a) + b q'(a) ϵ \\
r(a+b ϵ) &= r(a) + b r'(a) ϵ
\end{align*}
$$

**Proof**
For $q$ it follows immediately:
$$
\begin{align*}
q(a + b ϵ) &= f(g(a + b ϵ)) = f(g(a) + b g'(a) ϵ) \\
&= f(g(a)) + b g'(a) f'(g(a))ϵ = q(a) + b q'(a) ϵ.
\end{align*}
$$
For $r$ we have
$$
\begin{align*}
r(a + b ϵ) &= f(a+b ϵ )g(a+b ϵ )= (f(a) + b f'(a) ϵ)(g(a) + b g'(a) ϵ) \\
&= f(a)g(a) + b (f'(a)g(a) + f(a)g'(a)) ϵ = r(a) +b r'(a) ϵ.
\end{align*}
$$

∎

A simple corollary is that any function defined in terms of addition, multiplication, composition, etc.
of functions that are dual with differentiation will be differentiable via dual numbers.

**Example 2 (differentiating non-polynomial)**

Consider $f(x) =  \exp(x^2 + {\rm e}^{x})$ by evaluating on the duals:
$$
f(1 + ϵ) = \exp(1 + 2ϵ + {\rm e} + {\rm e} ϵ) =  \exp(1 + {\rm e}) + \exp(1 + {\rm e}) (2 + {\rm e}) ϵ
$$
and therefore we deduce that
$$
f'(1) = \exp(1 + {\rm e}) (2 + {\rm e}).
$$


## 3. Implementation as a special type


We now consider a simple implementation of dual numbers that works on general polynomials:

In [1]:
# Dual(a,b) represents a + b*ϵ
struct Dual{T}
    a::T
    b::T
end

# Dual(a) represents a + 0*ϵ
Dual(a::Real) = Dual(a, zero(a)) # for real numbers we use a + 0ϵ

# Allow for a + b*ϵ syntax
const ϵ = Dual(0, 1)

import Base: +, *, -, /, ^, zero, exp

# support polynomials like 1 + x, x - 1, 2x or x*2 by reducing to Dual
+(x::Real, y::Dual) = Dual(x) + y
+(x::Dual, y::Real) = x + Dual(y)
-(x::Real, y::Dual) = Dual(x) - y
-(x::Dual, y::Real) = x - Dual(y)
*(x::Real, y::Dual) = Dual(x) * y
*(x::Dual, y::Real) = x * Dual(y)

# support x/2 (but not yet division of duals)
/(x::Dual, k::Real) = Dual(x.a/k, x.b/k)

# a simple recursive function to support x^2, x^3, etc.
function ^(x::Dual, k::Integer)
    if k < 0
        error("Not implemented")
    elseif k == 1
        x
    else
        x^(k-1) * x
    end
end

# Algebraic operationds for duals
-(x::Dual) = Dual(-x.a, -x.b)
+(x::Dual, y::Dual) = Dual(x.a + y.a, x.b + y.b)
-(x::Dual, y::Dual) = Dual(x.a - y.a, x.b - y.b)
*(x::Dual, y::Dual) = Dual(x.a*y.a, x.a*y.b + x.b*y.a)

exp(x::Dual) = Dual(exp(x.a), exp(x.a) * x.b)

exp (generic function with 15 methods)

We can also try it on the two polynomials as above:

In [2]:
f = x -> 1 + x + x^2
g = x -> 1 + x/3 + x^2
f(ϵ).b, g(ϵ).b

(1, 0.3333333333333333)

The first example exactly computes the derivative, and the
second example is exact up to the last bit rounding!
It also works for higher order polynomials:

In [3]:
f = x -> 1 + 1.3x + 2.1x^2 + 3.1x^3
f(0.5 + ϵ).b - 5.725

8.881784197001252e-16

It is indeed "accurate to (roughly) 16-digits", the best we can hope for 
using floating point.

We can use this in "algorithms" as well as simple polynomials.
Consider the polynomial $1 + … + x^n$:

In [4]:
function s(n, x)
    ret = 1 + x # first two terms
    for k = 2:n
        ret += x^k
    end
    ret
end
s(10, 0.1 + ϵ).b

1.2345678999999998

This matches exactly the "true" (up to rounding) derivative:

In [5]:
sum((1:10) .* 0.1 .^(0:9))

1.2345678999999998

Finally, we can try the more complicated example:

In [6]:
f = x -> exp(x^2 + exp(x))
f(1 + ϵ)

Dual{Float64}(41.193555674716116, 194.362805189629)

What makes dual numbers so effective is that, unlike divided differences, they are not
prone to disasterous growth due to round-off errors.